In [1]:
import torch
from model.UNet3D import UNet3D
from utils import load_config, plot_overlay, plot_slices, plot_test
from dataloader import Dataset
from torch.utils.data import DataLoader, random_split
from train import Trainer, Tester
from logger import Logger
from postprocessing import calc_volumes

print(torch.__version__) 
print(torch.cuda.is_available())

if __name__ == "__main__":  # must be enabled for num_workers > 0
    config = load_config("config.json")
    dataset = Dataset(config)

    train_set, val_set, test_set = random_split(dataset=dataset,
                                                lengths=[0.7,0.1,0.2], 
                                                generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(dataset=train_set, 
                            batch_size=config["trainer"]["batch_size"],
                            shuffle=True
                            )
    
    val_loader = DataLoader(dataset=val_set, 
                            batch_size=1
                            )
    
    test_loader = DataLoader(dataset=test_set,
                            batch_size=1
                            )
    
    print(f"Train: {len(train_loader)}\nVal: {len(val_loader)}\nTest: {len(test_loader)}")

    model = UNet3D(in_channels=1, num_classes=2)
    myLogger = Logger(config=config, save=True)
    model.cuda()
    model.load_state_dict(torch.load(R"saved\20240227_172605 320x320x16 150e 0.0005 aug\weights.pth"))

c:\Users\Daniel\anaconda3\envs\py3109\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\Daniel\anaconda3\envs\py3109\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you

2.0.1+cu117
True
Augmentation done with (-10, 10). Total images: 74
Train: 52
Val: 8
Test: 14


In [ ]:
pd_data = calc_volumes(test_set, model)

In [1]:
import pandas as pd
df = pd.DataFrame(pd_data, columns=['Filename', 'Volume [mm^3]', 'Volume [L]', 'DSC (pre-resample)', 'DSC'])
df

NameError: name 'pd_data' is not defined

In [ ]:
import pandas as pd
from tabulate import tabulate

def pandas_df_to_markdown_table(df):
    # Dependent upon ipython
    # shamelessly stolen from https://stackoverflow.com/questions/33181846/programmatically-convert-pandas-dataframe-to-markdown-table
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    #display(Markdown(df_formatted.to_csv(sep="|", index=False)))
    return Markdown(df_formatted.to_csv(sep="|", index=False))
#     return df_formatted

def df_to_markdown(df, y_index=False):
    blob = tabulate(df, headers='keys', tablefmt='pipe')
    if not y_index:
        # Remove the index with some creative splicing and iteration
        return '\n'.join(['| {}'.format(row.split('|', 2)[-1]) for row in blob.split('\n')])
    return blob

In [ ]:
pandas_df_to_markdown_table(df)

In [ ]:
vobj.header

In [ ]:
15*24

In [3]:
from postprocessing import Volume
pred = model(test_set[0][0].unsqueeze(0).unsqueeze(0).cuda())
vobj = Volume(image=test_set[0][0], 
              mask=test_set[0][1],
              prediction=pred,
              header=test_set[0][2])

vobj.get_volume()
print(f"Shape after resampling: {vobj.prediction_mask.shape}")


torch.Size([160, 160, 16])
[0 1]
DSC (after resample): 0.9610176086425781
Voxels: 474671 
Volume 1 voxel: 14.062499999999622
Shape after resampling: torch.Size([160, 160, 90])
DSC: 0.9793073139184256


In [ ]:
vobj.prediction_mask.shape

In [4]:
import meshlib.mrmeshpy as mr
import meshlib.mrmeshnumpy as mrn

simpleVolume = mrn.simpleVolumeFrom3Darray(vobj.prediction_mask.numpy())
floatGrid = mr.simpleVolumeToDenseGrid(simpleVolume )
mesh = mr.gridToMesh(floatGrid , mr.Vector3f(0.1, 0.1, 0.1), 0.5)
mr.saveMesh(mesh, "mesh.stl" )

In [ ]:
test_set[0][2]

In [ ]:
640*640*17*(0.625*0.625*24)

In [ ]:
160*160*16*(2.5*2.5*25.5)

In [ ]:
import numpy as np
voxel_volume = vobj.get_new_spacings(test_set[0][2], test_set[0][1].shape)
print(voxel_volume)
print()
np.count_nonzero(test_set[0][1]==1)*6*(2.5*2.5*4)

In [5]:
import open3d as o3d
mesh = o3d.io.read_triangle_mesh("mesh.stl")
mesh = mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], window_name="STL", left=1000, top=200, width=800, height=650)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
plot_test(test_set[0][0], test_set[0][1], pred, mask_only=True, nr_slices=10)

In [ ]:
import numpy as np
np.unique(vobj.prediction_mask)